In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import openpyxl
from itertools import combinations, permutations
from collections import Counter

## 아이템 상호작용 행렬 만들기

1. 필요한 데이터 로딩

In [4]:
# TODO : 데이터 경로 설정 
raw_data = pd.read_excel("/opt/ml/input/data/raw_codimap/item/view/item.xlsx", engine='openpyxl')
df = raw_data[["id", "codi_id"]]
unique_codi_id = list(df["codi_id"].unique())
unique_id = list(df["id"].unique())
codi_group_by_codi = df.groupby("codi_id")["id"].apply(list)
codi_group_by_item = df.groupby("id")["codi_id"].apply(list)

In [5]:
df["id"].value_counts()

1551840    10
404949      9
2024232     5
1278214     5
1833230     5
           ..
1960520     1
838145      1
399948      1
1647181     1
1937003     1
Name: id, Length: 2381, dtype: int64

In [6]:
cnt = 0
for count in df["id"].value_counts() : 
    if count == 1 :
        cnt +=1

print(f"Number of total instance : {len(df)}")
print(f"Number of unique items : {len(unique_id)}")
print(f"Number of items which was only collected once : {cnt}")

Number of total instance : 2863
Number of unique items : 2381
Number of items which was only collected once : 2038


2. Pivot Table을 위한 데이터 처리 과정

In [6]:
df = pd.read_excel("/opt/ml/input/data/raw_codishop/codi_item_id.xlsx", engine='openpyxl')

dup = list()
gf = df.groupby(['id'])['item_id'].apply(list)
for val in tqdm(gf):
    val = sorted(val)
    comb_res = list(permutations(val, 2))
    dup.extend(comb_res)
Counter(dup).most_common()

100%|██████████| 412/412 [00:00<00:00, 244905.51it/s]


[((212880, 241111), 2),
 ((241111, 212880), 2),
 ((1131980, 1174440), 2),
 ((1174440, 1131980), 2),
 ((864554, 1137212), 2),
 ((1137212, 864554), 2),
 ((577636, 1222009), 2),
 ((1222009, 577636), 2),
 ((1141123, 1222009), 2),
 ((1222009, 1141123), 2),
 ((1292227, 1321881), 2),
 ((1321881, 1292227), 2),
 ((1365882, 1405150), 2),
 ((1405150, 1365882), 2),
 ((80701, 86559), 1),
 ((86559, 80701), 1),
 ((251483, 254485), 1),
 ((254485, 251483), 1),
 ((186610, 264922), 1),
 ((264922, 186610), 1),
 ((218237, 282220), 1),
 ((282220, 218237), 1),
 ((104239, 317826), 1),
 ((317826, 104239), 1),
 ((321088, 386408), 1),
 ((386408, 321088), 1),
 ((265767, 417045), 1),
 ((417045, 265767), 1),
 ((260755, 295623), 1),
 ((295623, 260755), 1),
 ((396038, 433275), 1),
 ((396038, 468975), 1),
 ((433275, 396038), 1),
 ((433275, 468975), 1),
 ((468975, 396038), 1),
 ((468975, 433275), 1),
 ((267464, 467795), 1),
 ((467795, 267464), 1),
 ((165980, 418341), 1),
 ((418341, 165980), 1),
 ((491406, 500038), 1),


3. 상호작용 행렬 만들기 & 데이터 내보내기

In [7]:
IIM = pd.DataFrame(columns=["id", "item", "count"])
_id = []
_item = []
_count = []
for pair, count in Counter(dup).most_common():
    _id.append(pair[0])
    _item.append(pair[1])
    _count.append(count)
    # IIM.pivot_table(index="id", columns="item", values="count").fillna(0)

_id = np.array(_id)
_item = np.array(_item)
_count = np.array(_count)

In [8]:
result = np.vstack([_id,_item,_count]).T
result

array([[ 212880,  241111,       2],
       [ 241111,  212880,       2],
       [1131980, 1174440,       2],
       ...,
       [2349427, 1820934,       1],
       [2349427, 1866814,       1],
       [2349427, 1999997,       1]])

In [9]:
IIM = pd.DataFrame(result, columns=["id", "item", "count"]).pivot_table(index="id", columns="item", values="count").fillna(0)
# TODO : 데이터 경로 설정
IIM.to_csv("/opt/ml/input/data/itemInteractionMatrix.csv")

4. Sparsity 점검

In [10]:
ones = 0
zeros = 0
for i in range(len(IIM)) :
    zeros += IIM.iloc[i].value_counts()[0]
    ones += IIM.iloc[i].value_counts()[1]
print(f"Sparsity of this Data is : {zeros/(ones+zeros):.4}")

KeyError: 1